# Анализ данных для книжного сервиса

Заказчик купил крупный сервис для чтения книг по подписке. Задача - проанализировать базу данных. В ней - информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задания**

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных**

Таблица `books` содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.


Таблица `authors` содержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

Таблица `publishers` содержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

Таблица `ratings` содержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах на книги:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text — текст обзора.

## Подключение к базе

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Обзор данных

Посмотрим на первые несколько строк каждой таблицы, для этого напишем функцию.

In [2]:
def request(table):
    query = '''
            SELECT *
            FROM {}
            LIMIT 3;
        '''.format(table)
    display(pd.io.sql.read_sql(query, con = engine))

In [3]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews', 'ratings']

for table in tables:
    print('Таблица', table)
    request(table)
    print()

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135



Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie



Таблица publishers


...

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5



Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...



Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


## Выполнение задач

**1. Посчитаем, сколько книг вышло после 1 января 2000 года.**

In [4]:
query = ''' 
            SELECT count(DISTINCT(title)) AS books
            FROM books
            WHERE date(publication_date) > '2001-01-01';
        '''
display(pd.io.sql.read_sql(query, con = engine))

,books
0,779


Всего 779 книг, это очень скромный показатель за 20 лет.

**2. Для каждой книги посчитаем количество обзоров и среднюю оценку.**

In [5]:
query = ''' 
            SELECT DISTINCT(b.title) AS book,
                   COUNT(re.review_id) AS total_reviews,
                   ROUND(AVG(ra.rating), 2) AS average_rating
            FROM books AS b
            LEFT JOIN reviews AS re ON b.book_id=re.book_id
            LEFT JOIN ratings AS ra ON re.book_id=ra.book_id
            GROUP BY b.title;
        '''
display(pd.io.sql.read_sql(query, con = engine))

,book,total_reviews,average_rating
0,Salem Falls,9,4.33
1,House of Leaves,16,4.75
2,I Am Legend,4,3.50
3,The Philosophy of Andy Warhol (From A to B and...,2,4.50
4,Undaunted Courage: The Pioneering First Missio...,4,4.00
...,...,...,...
994,Cold Mountain,32,3.63
995,The Bad Place,2,3.50
996,A Painted House,9,4.00
997,The Sinner (Rizzoli & Isles #3),4,4.50


У нас есть информация по обзорам и рейтингу для каждой книги, пользователям будет проще выбрать книгу на основе этих данных.

**3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.**

In [6]:
display(pd.io.sql.read_sql(query, con = engine))

,book,total_reviews,average_rating
0,Salem Falls,9,4.33
1,House of Leaves,16,4.75
2,I Am Legend,4,3.50
3,The Philosophy of Andy Warhol (From A to B and...,2,4.50
4,Undaunted Courage: The Pioneering First Missio...,4,4.00
...,...,...,...
994,Cold Mountain,32,3.63
995,The Bad Place,2,3.50
996,A Painted House,9,4.00
997,The Sinner (Rizzoli & Isles #3),4,4.50


In [7]:
query = '''       
           SELECT publisher
           FROM publishers
           WHERE publisher_id IN (SELECT publisher_id
                                  FROM books
                                  WHERE num_pages > 50
                                  GROUP BY publisher_id
                                  ORDER BY COUNT(book_id) DESC
                                  LIMIT 1);
        '''
display(pd.io.sql.read_sql(query, con = engine))

,publisher
0,Penguin Books


У нас есть информация о большом издательстве, с которым в перспективе можно наладить сотрудничество по поставке электронных книг.

**4. Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками.**

In [8]:
query = '''   
           SELECT a.author
           FROM books AS b
           LEFT JOIN authors AS a ON b.author_id=a.author_id
           WHERE book_id IN (SELECT book_id
                             FROM ratings
                             WHERE book_id IN (SELECT book_id
                                               FROM ratings
                                               GROUP BY book_id
                                               HAVING COUNT(rating_id) >= 50)
                             GROUP BY book_id
                             ORDER BY AVG(rating) DESC
                             LIMIT 1);
        '''
display(pd.io.sql.read_sql(query, con = engine))

,author
0,J.K. Rowling/Mary GrandPré


Эта информация может быть полезна для проведения рекламных кампаний.

**5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [9]:
query = '''   
           SELECT ROUND(AVG(review_id), 2)
           FROM reviews
           WHERE username IN (SELECT username
                              FROM ratings
                              GROUP BY username
                              HAVING COUNT(rating_id) > 50)        
        '''
display(pd.io.sql.read_sql(query, con = engine))

,round
0,1315.76


У сервиса есть очень активные пользователи, которые делают обзоры и ставят оценки. Это поможет другим пользователя в выборе книг.